In [ ]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Question 1**

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

botswana.tsv

О каждой из них мы знаем:

* сколько детей она родила (признак ceb)
* возраст (age)
* длительность получения образования (educ)
* религиозная принадлежность (religion)
* идеальное, по её мнению, количество детей в семье (idlnchld)
* была ли она когда-нибудь замужем (evermarr)
* возраст первого замужества (agefm)
* длительность получения образования мужем (heduc)
* знает ли она о методах контрацепции (knowmeth)
* использует ли она методы контрацепции (usemeth)
* живёт ли она в городе (urban)
* есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
* Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [ ]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [ ]:
raw['religion'].value_counts().sort_values(ascending=False)

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

**Question 2**

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [ ]:
nan_values = raw.dropna()
nan_values.shape

(1834, 15)

**Question 3**

In [ ]:
raw['nevermarr'] = [1 if raw.loc[i, 'evermarr'] == 0 else 0 for i in range(raw.shape[0])]
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [ ]:
raw.agefm[raw.agefm.isnull()] = 0
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [ ]:
del raw['evermarr']

In [ ]:
raw.heduc[raw.heduc.isnull() & raw.nevermarr.values == 1] = -1
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [ ]:
raw['heduc'].isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

**Question 4**

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [ ]:
raw['idlnchld_noans'] = 0
raw.loc[raw.idlnchld.isnull(), 'idlnchld_noans'] = 1

raw['heduc_noans'] = 0
raw.loc[raw.heduc.isnull(), 'heduc_noans'] = 1

raw['usemeth_noans'] = 0
raw.loc[raw.usemeth.isnull(), 'usemeth_noans'] = 1

raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [ ]:
raw.idlnchld[raw.idlnchld.isnull()] = -1
raw.heduc[raw.heduc.isnull()] = -2
raw.usemeth[raw.usemeth.isnull()] = -1

In [ ]:
raw = raw.dropna()
array_size = raw.shape[0] * raw.shape[1]
array_size

78264

**Question 5**

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2? Округлите до трёх знаков после десятичной точки.

In [ ]:
reg_m = smf.ols('ceb ~ ' + ' + '.join(raw.columns[1:]), data=raw)
fitted_m = reg_m.fit()
fitted_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        18:09:05   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans             0.6539      0.153      4.286      0.000       0.355       0.953
heduc_noans               -0.8724      0.145     -6.026      0.000      -1.156      -0.589
usemeth_noans              0.7652      0.196      3.910      0.000       0.382       1.149
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Question 6**

Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [ ]:
raw.religion.value_counts()

spirit        1838
other         1076
protestant     989
catholic       445
Name: religion, dtype: int64

**Question 7**

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [ ]:
sms.het_breuschpagan(fitted_m.resid, fitted_m.model.exog)[1]

1.1452927633445016e-225

In [ ]:
reg_m2 = smf.ols('ceb ~ ' + ' + '.join(raw.columns[1:]), data=raw)
fitted_m2 = reg_m2.fit(cov_type='HC1')
fitted_m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        17:55:21   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.266     -3.863      0.000      -1.547      -0.506
religion[T.other]         -0.0830      0.078     -1.067      0.286      -0.235       0.069
religion[T.protestant]    -0.0149      0.078     -0.192      0.848      -0.167       0.137
religion[T.spirit]        -0.0191      0.071     -0.268      0.789      -0.159       0.121
age                        0.1703      0.004     38.627      0.000       0.162       0.179
educ                      -0.0724      0.007     -9.924      0.000      -0.087      -0.058
idlnchld                   0.0760      0.015      5.236      0.000       0.048       0.104
knowmeth                   0.5564      0.174      3.190      0.001       0.215       0.898
usemeth                    0.6473      0.052     12.478      0.000       0.546       0.749
agefm                     -0.0604      0.010     -6.174      0.000      -0.080      -0.041
heduc                     -0.0551      0.009     -6.126      0.000      -0.073      -0.037
urban                     -0.2137      0.046     -4.667      0.000      -0.303      -0.124
electric                  -0.2685      0.072     -3.732      0.000      -0.410      -0.128
radio                     -0.0235      0.053     -0.446      0.656      -0.127       0.080
tv                        -0.1451      0.082     -1.766      0.077      -0.306       0.016
bicycle                    0.2139      0.048      4.412      0.000       0.119       0.309
nevermarr                 -2.2393      0.202    -11.082      0.000      -2.635      -1.843
idlnchld_noans             0.6539      0.216      3.029      0.002       0.231       1.077
heduc_noans               -0.8724      0.191     -4.556      0.000      -1.248      -0.497
usemeth_noans              0.7652      0.213      3.590      0.000       0.347       1.183
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""


**Question 8**


Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [ ]:
reg_m3 = smf.ols( 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle \
                  + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
                  data=raw)
fitted_m3 = reg_m3.fit()
fitted_m3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        17:56:16   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.393      0.000      -1.459      -0.681
age                0.1702      0.003     52.271      0.000       0.164       0.177
educ              -0.0729      0.007    -10.285      0.000      -0.087      -0.059
idlnchld           0.0770      0.011      7.042      0.000       0.056       0.098
knowmeth           0.5610      0.121      4.628      0.000       0.323       0.799
usemeth            0.6516      0.048     13.537      0.000       0.557       0.746
agefm             -0.0606      0.007     -9.240      0.000      -0.073      -0.048
heduc             -0.0573      0.008     -7.186      0.000      -0.073      -0.042
urban             -0.2190      0.047     -4.682      0.000      -0.311      -0.127
electric          -0.3207      0.070     -4.584      0.000      -0.458      -0.184
bicycle            0.2046      0.049      4.154      0.000       0.108       0.301
nevermarr         -2.2501      0.148    -15.231      0.000      -2.540      -1.961
idlnchld_noans     0.6565      0.152      4.310      0.000       0.358       0.955
heduc_noans       -0.8853      0.145     -6.122      0.000      -1.169      -0.602
usemeth_noans      0.7732      0.196      3.955      0.000       0.390       1.156
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
sms.het_breuschpagan(fitted_m3.resid, fitted_m3.model.exog)[1]

1.119745889653916e-228

In [ ]:
reg_m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle \
                  + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
                  data=raw)
fitted_m4 = reg_m4.fit(cov_type='HC1')
fitted_m4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        17:57:07   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8853      0.191     -4.638      0.000      -1.259      -0.511
usemeth_noans      0.7732      0.212      3.639      0.000       0.357       1.190
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [ ]:
reg_m2.fit().compare_f_test(reg_m4.fit())

(0.9192357784628133, 0.46723055472768693, 5.0)

**Question 9**


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [ ]:
reg_m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle \
                + nevermarr + idlnchld_noans + heduc_noans', 
                data=raw)
fitted_m5 = reg_m5.fit()
fitted_m5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        17:59:50   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.897      0.000      -1.590      -0.796
age                0.1776      0.003     54.368      0.000       0.171       0.184
educ              -0.0560      0.007     -7.862      0.000      -0.070      -0.042
idlnchld           0.0705      0.011      6.317      0.000       0.049       0.092
knowmeth           0.8739      0.121      7.203      0.000       0.636       1.112
agefm             -0.0649      0.007     -9.716      0.000      -0.078      -0.052
heduc             -0.0521      0.008     -6.411      0.000      -0.068      -0.036
urban             -0.1866      0.048     -3.917      0.000      -0.280      -0.093
electric          -0.3218      0.071     -4.505      0.000      -0.462      -0.182
bicycle            0.1979      0.050      3.935      0.000       0.099       0.296
nevermarr         -2.3625      0.150    -15.707      0.000      -2.657      -2.068
idlnchld_noans     0.5266      0.155      3.393      0.001       0.222       0.831
heduc_noans       -0.7947      0.147     -5.389      0.000      -1.084      -0.506
==============================================================================
Omnibus:                      250.641   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              936.515
Skew:                          -0.158   Prob(JB):                    4.35e-204
Kurtosis:                       5.251   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
reg_m4.fit().compare_f_test(reg_m5.fit())

(92.89058230109622, 3.1552009480426394e-40, 2.0)

**Question 10**

Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [ ]:
fitted_m4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        18:00:54   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8853      0.191     -4.638      0.000      -1.259      -0.511
usemeth_noans      0.7732      0.212      3.639      0.000       0.357       1.190
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""